# Projeto E-Commerce: Identificar os perfis dos consumidores

### Tarefa: Segmente usuários com base em seus perfis de consumidor

#### Objetivo: Segmentar os usuários com base no histórico de compras, para desenvolver recomendações para a equipe de marketing sobre ofertas mais personalizadas para diferentes perfis

#### 1. Importando bibliotecas

In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns 
import datetime as dt 

#### 2. Leitura dos dados e revisão bibliográfica sobre o tema

In [ ]:
df = pd.read_csv('ecommerce_dataset_us.csv', sep='\t')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
224970,556580,22624,IVORY KITCHEN SCALES,2,06/11/2019 14:20,8.50,17719.0
163678,550621,84596J,MIXED NUTS LIGHT GREEN BOWL,2,04/17/2019 13:41,0.42,13263.0
67207,541827,22971,QUEENS GUARD COFFEE MUG,1,01/19/2019 17:05,4.96,NaN
512131,579509,23111,PARISIENNE SEWING BOX,1,11/27/2019 16:44,12.50,12963.0
1550,536544,22314,OFFICE MUG WARMER CHOC+BLUE,1,11/29/2018 14:32,5.91,NaN
153071,549729,21498,RED RETROSPOT WRAP,25,04/09/2019 16:17,0.42,13634.0
84332,543382,21877,HOME SWEET HOME MUG,6,02/05/2019 16:21,1.25,14030.0
88792,543816,22676,FRENCH BLUE METAL DOOR SIGN 1,1,02/11/2019 15:26,1.25,17841.0
155072,549964,20829,GLITTER HANGING BUTTERFLY STRING,8,04/11/2019 14:12,2.10,13122.0
342543,566872,23004,TRAVEL CARD WALLET PANTRY,24,09/13/2019 13:28,0.42,16729.0


In [11]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,11/29/2018 08:26,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,11/29/2018 08:26,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,11/29/2018 08:26,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,11/29/2018 08:26,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,11/29/2018 08:26,3.39,17850.0


##### Revisão bibliográficas e abordagem sobre o tema:

* Segmentação de clientes: guia completo:
Este guia detalha como a segmentação de clientes auxilia equipes de marketing e vendas a prever metas futuras com maior precisão, permitindo um planejamento mais eficaz de campanhas. [MONDAY.COM](https://monday.com/blog/pt/vendas-crm/segmentacao-de-clientes-para-vendas-e-marketing-b2b/?utm_source=chatgpt.com)

* Segmentação do Ciclo de Vida do Cliente para E-commerce:
O artigo aborda o processo de agrupar clientes com base em sua posição no relacionamento com a empresa, permitindo personalizar interações e melhorar a experiência do cliente. [CICLO E-COMMERCE](https://cicloecommerce.com.br/segmentacao-do-ciclo-de-vida-do-cliente-para-e-commerce/?utm_source=chatgpt.com)

* Segmentação de Clientes usando K-Means Clustering:
Este artigo no Medium explora como o clustering K-Means pode ser utilizado para segmentar clientes com base em seus comportamentos de compra, oferecendo uma visão prática da aplicação desse algoritmo. 
[MEDIUM](https://medium.com/%40larixgomex/segmenta%C3%A7%C3%A3o-de-clientes-usando-k-means-clustering-739ca0168e2b)

* Segmentação de Clientes com K-Means
Neste artigo do LinkedIn, o autor compartilha sua experiência em um projeto de segmentação de clientes, utilizando a análise RFM e o algoritmo K-Means, destacando insights e resultados obtidos. 
[LINKEDIN](https://www.linkedin.com/pulse/segmenta%C3%A7%C3%A3o-de-clientes-com-k-means-estevao-smith-trmqf/)

* Qual o melhor grupo de clientes? — RFM Análise (K-Means + EDA)
Este artigo da Dataside aborda a análise RFM combinada com o K-Means e a Análise Exploratória de Dados (EDA), discutindo como identificar os melhores grupos de clientes para estratégias de marketing mais eficazes. 
[DATASIDE](https://www.dataside.com.br/dataside-community/a-i-e-machine-learning/qual-o-melhor-grupo-de-clientes-rfm-analise-k-means-eda?utm_source=chatgpt.com)

* Segmentação de Clientes com RFM em Python
Este artigo no Medium detalha como implementar a análise RFM utilizando Python, oferecendo um guia passo a passo para segmentar clientes com base em seu comportamento de compra. 
[MEDIUM](https://medium.com/%40larixgomex/segmenta%C3%A7%C3%A3o-de-clientes-com-rfm-em-python-3a97e534ffa1)

* Segmentação de Clientes: conheça a análise RFM
Neste artigo do LinkedIn, o autor introduz a análise RFM como uma técnica para segmentar clientes, discutindo sua importância e aplicação no mercado de varejo. 
[LINKEDIN](https://www.linkedin.com/pulse/segmenta%C3%A7%C3%A3o-de-clientes-conhe%C3%A7a-an%C3%A1lise-rfm-fabio-innocente-alves/)



Baseado na revisão bibliográfica a idéia de abordagem sobre o projeto seria:

* Análise RFM primeiro:
    + Calcular as métricas RFM (Recência, Frequência e Valor Monetário) para cada cliente, essa é uma forma de segmentação da base de clientes com base em quão engajados com o seu negócio eles estão.
    + Normalizar os valores para que fiquem na mesma escala.
    + Analisar distribuições e possíveis outliers.

* Aplicação do K-Means depois:
    + Utilizar os scores RFM como variáveis para o modelo de clusterização.
    + Escolher o número ideal de clusters (usando o método Ward, metodo cotovelo ou Silhouette score).
    + Analisar e Interpretar os clusters e validar se fazem sentido para o negócio.
    + Dessa forma, o K-Means consegueria identificar diferentes segmentos de consumidores com base em seus comportamentos de compra, permitindo insights mais precisos para estratégias de marketing.

##### Plano de análise de perfis: 

* Verificar o valor médio do RFM por cluster, onde os resultados esperados podem ser: 
    + Clientes principais ou clientes ouros: clientes com carteira aberta, muito valor monetário e que compram frequentemente (fidelização e upselling)
    + Clientes casuais ou pratas: clientes que compram com boa frequencia mas com baixo valor monetário (aumento de gasto por transação, importantes pro volumes de vendas)
    + Clientes esporádicos ou bronze: clientes que aparecem raramente e não se ouvem mais falar, podem também ser clientes com baixo valor monetário(desconto segunda ou terceira compra, medidas para cativá-los)

* Perguntas que podem ser respondidas após análise de perfis: 
    + Por que isso é importante? O que podemos obter com isso?
    + Quem são seus melhores clientes?
    + Quais dos seus clientes podem contribuir para o churn?
    + Quem tem o potencial de se tornar um cliente valioso?
    + Quais dos nossos clientes podem ser retidos?
    + Quais dos nossos clientes têm mais probabilidade de responder às campanhas de engajamento?
    + Quais tipos de campanhas de engajamentos podemos oferecer para cada cluster?


#### 3. Análise de qualidade e limpeza dos dados

##### Formato dos dados e nome das colunas 

##### Valores ausentes 

##### Valores duplicados

#### 4. EDA

##### Verificando distribuição dos dados 

##### Verificando outliers 

##### Verificando relação das variáveis entre si e colunas interessantes para segmentação construção de modelo de agrupamento 

#### 5. Segmentação de perfis de usuários 

##### Engenhenharia de características e calculo de métricas RFM sobre os usuários 

##### Novo DF com RFM e análise de distribuições e possíveis outliers

##### Normalização de valores para que fiquem na mesma escala.

##### Escolhendo o número ideal de clusters usando o método Ward sobre os scores RFM

##### Treinamento de ML de agrupamento K-Means baseado em scores RFM como variáveis para o modelo de clusterização.

##### Analisando número ideal de clusters com método cotovelo e silhouette score 
plot_model(kmeans, plot = 'elbow')

##### Interpretando os clusters e validando se fazem sentido para o negócio.

#### 6. Análise de clusters criados

##### Distribuição dos dados por perfil
plot de cada característica para diferenciar entre os perfis 

Análises devem ser feitas em cima do objetivo de gerar recomendações para mkt  

#### 7. Conclusão e recomendações 